# ANIME RECOMMENDER SYSTEM - HYBRID FILTERING - TWO TOWER - CASE 1

In [ ]:
# basic library
import numpy as np 
import pandas as pd  
import warnings
warnings.filterwarnings('ignore')
import pickle
from PIL import Image

In [2]:
# load data
file = open('dataset/processed_dataset/merged/merged_df_1.pkl', 'rb')
merged_df = pickle.load(file)

In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512740 entries, 0 to 1512739
Data columns (total 87 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   user_id               1512740 non-null  int64  
 1   user_score            1512740 non-null  float64
 2   num_watched_episodes  1512740 non-null  float64
 3   created_at            1512740 non-null  float64
 4   updated_at            1512740 non-null  float64
 5   anime_id              1512740 non-null  float64
 6   score                 1512740 non-null  float64
 7   members               1512740 non-null  float64
 8   type                  1512740 non-null  int64  
 9   studio                1512740 non-null  int64  
 10  episode_count         1512740 non-null  float64
 11  Action                1512740 non-null  int64  
 12  Adult Cast            1512740 non-null  int64  
 13  Adventure             1512740 non-null  int64  
 14  Anthropomorphic       1512740 non-

## train test split

In [4]:
from sklearn.utils import shuffle
merged_df = shuffle(merged_df)

In [5]:
from sklearn.model_selection import train_test_split
y = merged_df.user_score
x = merged_df.drop('user_score', axis=1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=0)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1058918, 86)
(1058918,)
(453822, 86)
(453822,)


In [7]:
user_id_train = x_train.user_id
anime_id_train = x_train.anime_id
anime_content_train = x_train.drop(['user_id', 'anime_id', 'num_watched_episodes','created_at','updated_at'], axis=1)
rating_content_train = x_train[['num_watched_episodes','created_at','updated_at']]

In [16]:
user_id_test = x_test.user_id
anime_id_test = x_test.anime_id
anime_content_test = x_test.drop(['user_id', 'anime_id', 'num_watched_episodes','created_at','updated_at'], axis=1)
rating_content_test = x_test[['num_watched_episodes','created_at','updated_at']]

In [9]:
print(user_id_train.shape)
print(anime_id_train.shape)
print(anime_content_train.shape)
print(rating_content_train.shape)

(1058918,)
(1058918,)
(1058918, 81)
(1058918, 3)


## model building

In [10]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Sequential

In [11]:
# #func api
# user_id = keras.Input(shape=(1,), name="in")
# x = tf.keras.layers.Embedding(1058918, 32, input_length=1, embeddings_initializer="RandomNormal",embeddings_regularizer=l2(0))(user_id)
# x = Flatten()(x)
# x = Dense(8, activation="relu")(x)
# x = Dense(4, activation="relu")(x)
# encoder_output = Dense(4, activation="relu")(x)

# anime_id = keras.Input(shape=(1,), name="in2")
# x2 = tf.keras.layers.Embedding(1058918, 32, input_length=1, embeddings_initializer="RandomNormal",embeddings_regularizer=l2(0))(anime_id)
# x2 = Flatten()(x2)
# anime_content = keras.Input(shape=(84,), name="in2_2")
# x2_1 = Dense(128, activation="relu")(anime_content)
# x2_2 =tf.keras.layers.Concatenate(axis=1)([x2, x2_1])

# x2_2 = Dense(8, activation="relu")(x2_2)
# x2_2 = Dense(4, activation="relu")(x2_2)
# encoder_output_2 = Dense(4, activation="relu")(x2_2)

# dotted = tf.keras.layers.Dot(axes=-1)([encoder_output, encoder_output_2])

# model = keras.Model(inputs=[user_id, anime_id, anime_content], outputs=dotted, name="model")

In [12]:
#func api
user_id = keras.Input(shape=(1,), name="in")
x = tf.keras.layers.Embedding(1058918, 32, input_length=1, embeddings_initializer="RandomNormal",embeddings_regularizer=l2(0))(user_id)
x = Flatten()(x)
user_content = keras.Input(shape=(3,), name="in1_2")
x1_1 = Dense(128, activation="relu")(user_content)
x1_2 =tf.keras.layers.Concatenate(axis=1)([x, x1_1])

x1_2 = Dense(8, activation="relu")(x1_2)
x1_2 = Dense(4, activation="relu")(x1_2)
encoder_output = Dense(4, activation="relu")(x1_2)

anime_id = keras.Input(shape=(1,), name="in2")
x2 = tf.keras.layers.Embedding(1058918, 32, input_length=1, embeddings_initializer="RandomNormal",embeddings_regularizer=l2(0))(anime_id)
x2 = Flatten()(x2)
anime_content = keras.Input(shape=(81,), name="in2_2")
x2_1 = Dense(128, activation="relu")(anime_content)
x2_2 =tf.keras.layers.Concatenate(axis=1)([x2, x2_1])

x2_2 = Dense(8, activation="relu")(x2_2)
x2_2 = Dense(4, activation="relu")(x2_2)
encoder_output_2 = Dense(4, activation="relu")(x2_2)

dotted = tf.keras.layers.Dot(axes=-1)([encoder_output, encoder_output_2])

model = keras.Model(inputs=[user_id, user_content, anime_id, anime_content], outputs=dotted, name="model")

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 in (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 in2 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 32)        33885376    ['in[0][0]']                     
                                                                                                  
 in1_2 (InputLayer)             [(None, 3)]          0           []                               
                                                                                              

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, weight_decay=1e-4)

model.compile(
    loss='mean_squared_error', #mse
    optimizer=optimizer,
    metrics=["mean_squared_error"], #mse 
)

history = model.fit([user_id_train, rating_content_train, anime_id_train, anime_content_train], y_train, batch_size=8192, epochs=10, validation_split=0.2)

Epoch 1/10
104/104 [==============================] - 131s 1s/step - loss: 0.0993 - mean_squared_error: 0.0993 - val_loss: 0.0339 - val_mean_squared_error: 0.0339
Epoch 2/10
104/104 [==============================] - 119s 1s/step - loss: 0.0279 - mean_squared_error: 0.0279 - val_loss: 0.0538 - val_mean_squared_error: 0.0538
Epoch 3/10
104/104 [==============================] - 121s 1s/step - loss: 0.0233 - mean_squared_error: 0.0233 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch 4/10
104/104 [==============================] - 115s 1s/step - loss: 0.0190 - mean_squared_error: 0.0190 - val_loss: 0.0191 - val_mean_squared_error: 0.0191
Epoch 5/10
104/104 [==============================] - 116s 1s/step - loss: 0.0187 - mean_squared_error: 0.0187 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch 6/10
104/104 [==============================] - 115s 1s/step - loss: 0.0185 - mean_squared_error: 0.0185 - val_loss: 0.0189 - val_mean_squared_error: 0.0189
Epoch 7/10
104/104 [==

In [17]:
predictions = model.predict([user_id_test, rating_content_test, anime_id_test, anime_content_test])

14182/14182 [==============================] - 14s 950us/step


In [18]:
print(type(predictions))

<class 'numpy.ndarray'>


In [19]:
from sklearn.metrics import mean_squared_error
print("MSE",mean_squared_error(y_test,predictions))

MSE 0.020436142107635658


In [20]:
from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test,predictions))

MAE 0.10757600790541634


In [21]:
from sklearn.metrics import r2_score
print("r-squared",r2_score(y_test,predictions))

r-squared 0.3727713853525718


In [22]:
from sklearn.metrics import mean_squared_error
print("RMSE",mean_squared_error(y_test,predictions, squared=False))

RMSE 0.1429550352650639


In [23]:
test_arr = predictions.ravel()
pred_arr = pd.DataFrame(test_arr, columns = ['pred'])

In [24]:
y_test_df = y_test.to_frame()

In [25]:
dat1 = pred_arr.reset_index(drop=True)
dat2 = y_test_df.reset_index(drop=True)

In [26]:
dat_1_2 = dat1.join(dat2)

In [27]:
dat_1_2

,pred,user_score
0,0.666939,0.777778
1,0.731643,0.777778
2,0.820035,0.666667
3,0.684945,0.777778
4,0.763537,0.666667
...,...,...
453817,0.514691,0.555556
453818,0.638451,0.777778
453819,0.609294,0.333333
453820,0.474019,0.222222


In [28]:
model.save('model/model_impl.h5')

In [29]:
user_id_test

1076833    14333
668168      8062
227396      2699
546016      6625
538922      6551
           ...  
1031625    13715
1382985    18328
1172547    15636
97633       1137
152041      1804
Name: user_id, Length: 453822, dtype: int64

In [30]:
anime_id_test

1076833     3901.0
668168      1498.0
227396     32542.0
546016      3858.0
538922     17121.0
            ...   
1031625     3016.0
1382985     3785.0
1172547    36873.0
97633        903.0
152041      3652.0
Name: anime_id, Length: 453822, dtype: float64

In [31]:
anime_content_test

,score,members,type,studio,episode_count,Action,Adult Cast,Adventure,Anthropomorphic,Avant Garde,...,Super Power,Supernatural,Survival,Suspense,Team Sports,Time Travel,Vampire,Video Game,Visual Arts,Workplace
1076833,0.889133,0.048063,3,107,0.000981,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
668168,0.780461,0.027775,4,267,0.003925,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
227396,0.829857,0.198872,4,665,0.003925,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
546016,0.737651,0.000836,4,770,0.029768,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
538922,0.804610,0.018809,0,137,0.000327,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031625,0.888035,0.008949,0,357,0.000327,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1382985,0.837541,0.112007,0,354,0.000327,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1172547,0.762898,0.038085,4,315,0.003271,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
97633,0.743139,0.034414,0,789,0.000327,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
